# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("./output_data/weather_output.csv")
weather_df = df.drop(columns=["Unnamed: 0"])
weather_df.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,normal,20,US,1608353360,86,40.51,-88.99,1.67,8.20
1,ribeira grande,95,PT,1608353360,78,38.52,-28.70,16.14,4.57
2,punta arenas,97,CL,1608353360,76,-53.15,-70.92,9.00,3.60
3,grand gaube,4,MU,1608353185,72,-20.01,57.66,27.22,3.13
4,vardo,70,NO,1608353361,94,70.37,31.11,-5.26,2.36


In [4]:
# Transform from celsius to fahrenheit
Max_Temp_F = [(x*1.8) + 32 for x in weather_df["Max Temp"]]

In [5]:
# Add to DataFrame as a new column
weather_df["Max Temp (F)"] = Max_Temp_F
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Max Temp (F)
0,normal,20,US,1608353360,86,40.51,-88.99,1.67,8.20,35.006
1,ribeira grande,95,PT,1608353360,78,38.52,-28.70,16.14,4.57,61.052
2,punta arenas,97,CL,1608353360,76,-53.15,-70.92,9.00,3.60,48.200
3,grand gaube,4,MU,1608353185,72,-20.01,57.66,27.22,3.13,80.996
4,vardo,70,NO,1608353361,94,70.37,31.11,-5.26,2.36,22.532


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Humidity Heatmap

weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)
locations = weather_df[["Lat", "Lng"]].astype(float)

# Add heatmap figure to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:

sum_weather_df = weather_df

# A max temperature lower than 80 degrees but higher than 70
sum_weather_df = sum_weather_df.loc[(sum_weather_df["Max Temp (F)"] < 80) & (sum_weather_df["Max Temp (F)"] > 70)]

# Wind speed less than 10 mph
sum_weather_df = sum_weather_df.loc[sum_weather_df["Wind Speed"] < 10]

# Zero cloudiness
sum_weather_df = sum_weather_df.loc[sum_weather_df["Cloudiness"] == 0]

sum_weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Max Temp (F)
85,rikitea,0,PF,1608353199,77,-23.12,-134.97,25.62,9.49,78.116
155,prieska,0,ZA,1608353407,42,-29.66,22.75,26.32,1.06,79.376
225,puri,0,IN,1608353284,60,19.80,85.85,22.00,5.10,71.600
237,esperance,0,AU,1608353432,31,-33.87,121.90,26.00,2.10,78.800
270,cockburn town,0,TC,1608353441,80,21.46,-71.14,25.53,5.96,77.954


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Hotel Map

hotel_df = sum_weather_df

# Set parameters to search hotels with 5000 meters
params = {
    "key": g_key,
    "radius": 5000,
    "keyword": "hotel",
    }

for index, row in hotel_df.iterrows():
    # Get lat, lng from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Create url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(url, params=params).json()
    print(response)
    # Extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError  ):
        print("Result not found... Skipping.")
        
    print("------------")

Retrieving Results for Index 85: rikitea.
{'error_message': 'You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
Result not found... Skipping.
------------
Retrieving Results for Index 155: prieska.
{'error_message': 'You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
Result not found... Skipping.
------------
Retrieving Results for Index 225: puri.
{'error_message': 'You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started', 'html_attributions': [], 'results': [], 'status': 'REQUE

In [12]:
# Store hotel result into a DataFrame
hotel_df.to_csv("./output_data/hotel_output.csv")
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Max Temp (F)
85,rikitea,0,PF,1608353199,77,-23.12,-134.97,25.62,9.49,78.116
155,prieska,0,ZA,1608353407,42,-29.66,22.75,26.32,1.06,79.376
225,puri,0,IN,1608353284,60,19.80,85.85,22.00,5.10,71.600
237,esperance,0,AU,1608353432,31,-33.87,121.90,26.00,2.10,78.800
270,cockburn town,0,TC,1608353441,80,21.46,-71.14,25.53,5.96,77.954


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [13]:
# Add marker layer ontop of heat map
# Add marker layer of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
symbol_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)
fig.add_layer(symbol_layer)
# Display Map
fig


NameError: name 'hotel_info' is not defined